In [1]:
import pandas as pd
import numpy as np

In [33]:
grouped_farms = pd.read_csv('../Datos/Procesados/FarmDatos.csv')
grouped_farms = grouped_farms[grouped_farms['Año Censal'] == 2018]
grouped_farms.head()

,Año Censal,Entidad,Municipio,UE Unidades económicas,A131A Valor agregado censal bruto (millones de pesos),Q000A Acervo total de activos fijos (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A121A Consumo intermedio (millones de pesos),A700A Total de gastos (millones de pesos),Q000B Depreciación total de activos fijos (millones de pesos),...,"J203A Total de sueldos al personal administrativo, contable y de dirección (millones de pesos)",Q000C Compras totales de activos fijos (millones de pesos),Q000D Ventas totales de activos fijos (millones de pesos),P030B Total de inventario final de productos en proceso (millones de pesos),A534A Tasa de rentabilidad promedio (Porcentaje),A231A Margen bruto de operación (Porcentaje),A436A Ingresos por suministro de bienes y servicios por persona ocupada (Pesos),A171A Remuneración media por persona ocupada remunerada (Pesos),J000A Total de remuneraciones (millones de pesos),A221A Formación bruta de capital fijo (millones de pesos)
3213,2018,aguascalientes,aguascalientes,330,619.236,149.861,2298.195,153.257,1727.476,16.386,...,14.815,9.847,0.482,0.0,47.478,37.816,2223110.194,156114.784,135.348,9.365
3214,2018,aguascalientes,asientos,9,3.464,0.858,9.816,0.583,6.554,0.170,...,0.000,0.257,0.045,0.0,39.590,28.362,377538.462,35214.286,0.493,0.212
3215,2018,aguascalientes,calvillo,14,20.783,5.208,73.246,3.323,56.214,1.162,...,0.554,0.177,0.008,0.0,38.301,32.062,1589908.235,185734.694,6.806,0.169
3216,2018,aguascalientes,cosio,3,0.446,0.260,1.019,0.119,0.537,0.025,...,0.000,0.000,0.000,0.0,46.830,31.894,145571.429,39250.000,0.157,0.000
3217,2018,aguascalientes,jesus maria,23,35.730,25.588,150.711,9.955,122.426,2.510,...,3.603,3.757,0.235,0.0,23.326,20.758,1247541.116,201817.241,16.625,3.522


In [34]:
# Calcular indicadores
grouped_farms['ROE'] = (grouped_farms['A231A Margen bruto de operación (Porcentaje)'] / 100) * grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)']
grouped_farms['ROA'] = (grouped_farms['A131A Valor agregado censal bruto (millones de pesos)'] / grouped_farms['Q000A Acervo total de activos fijos (millones de pesos)']) * 100
grouped_farms['Rentabilidad sobre ingresos'] = (grouped_farms['A231A Margen bruto de operación (Porcentaje)'] / 100) * grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)'] / grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)'] * 100
grouped_farms['Gastos operativos'] = grouped_farms['A700A Total de gastos (millones de pesos)'] / grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)'] * 100
grouped_farms['Gastos por farmacia'] = grouped_farms['A700A Total de gastos (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Ingresos por farmacia'] = grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Remuneración media'] = grouped_farms['J000A Total de remuneraciones (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Gasto en publicidad por farmacia'] = grouped_farms['K810A Gastos por publicidad (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Inventario total'] = (grouped_farms['P000A Total de inventario inicial (millones de pesos)'] + grouped_farms['P000B Total de inventario final (millones de pesos)']) / 2
grouped_farms['Valor agregado por farmacia'] = grouped_farms['A131A Valor agregado censal bruto (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Variación de existencias'] = grouped_farms['P000C Variación total de existencias (millones de pesos)'] / grouped_farms['M000A Total de ingresos por suministro de bienes y servicios (millones de pesos)'] * 100
grouped_farms['Depreciación por farmacia'] = grouped_farms['Q000B Depreciación total de activos fijos (millones de pesos)'] / grouped_farms['UE Unidades económicas']
grouped_farms['Eficiencia en costos'] = (grouped_farms['A121A Consumo intermedio (millones de pesos)'] / grouped_farms['A131A Valor agregado censal bruto (millones de pesos)']) * 100


In [35]:
cluster_df = pd.read_csv('../Datos/Resultados/Clusters.csv')
cluster_df.head()

,Entidad,Municipio,Cluster
0,aguascalientes,aguascalientes,40
1,aguascalientes,asientos,21
2,aguascalientes,calvillo,42
3,aguascalientes,cosio,28
4,aguascalientes,el llano,2


In [36]:
#EFICIENCIA
# Define weights for each parameter
weights = {
    'ROE': 0.25,                       # Weight for Return on Equity
    'ROA': 0.20,                       # Weight for Return on Assets
    'Rentabilidad sobre ingresos': 0.15, # Weight for Profitability over Income
    'Gastos operativos': -0.10,        # Weight for Operating Expenses (negative, as lower is better)
    'Gastos por farmacia': -0.05,      # Weight for Expenses per Pharmacy (negative)
    'Ingresos por farmacia': 0.15,     # Weight for Income per Pharmacy
    'Remuneración media': 0.05,        # Weight for Average Salary
    'Gasto en publicidad por farmacia': -0.05, # Weight for Advertising Expense per Pharmacy (negative)
    'Valor agregado por farmacia': 0.05, # Weight for Value Added per Pharmacy
    'Eficiencia en costos': 0.10        # Weight for Cost Efficiency
}

# Calculate efficiency as a weighted sum
grouped_farms['Eficiencia'] = (
    grouped_farms['ROE'] * weights['ROE'] +
    grouped_farms['ROA'] * weights['ROA'] +
    grouped_farms['Rentabilidad sobre ingresos'] * weights['Rentabilidad sobre ingresos'] +
    grouped_farms['Gastos operativos'] * weights['Gastos operativos'] +
    grouped_farms['Gastos por farmacia'] * weights['Gastos por farmacia'] +
    grouped_farms['Ingresos por farmacia'] * weights['Ingresos por farmacia'] +
    grouped_farms['Remuneración media'] * weights['Remuneración media'] +
    grouped_farms['Gasto en publicidad por farmacia'] * weights['Gasto en publicidad por farmacia'] +
    grouped_farms['Valor agregado por farmacia'] * weights['Valor agregado por farmacia'] +
    grouped_farms['Eficiencia en costos'] * weights['Eficiencia en costos']
)


In [37]:
merged_df = pd.merge(grouped_farms, cluster_df, on=['Entidad', 'Municipio'], how='left')
merged_df.sort_values('Eficiencia', ascending=False)

,Año Censal,Entidad,Municipio,UE Unidades económicas,A131A Valor agregado censal bruto (millones de pesos),Q000A Acervo total de activos fijos (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A121A Consumo intermedio (millones de pesos),A700A Total de gastos (millones de pesos),Q000B Depreciación total de activos fijos (millones de pesos),...,Ingresos por farmacia,Remuneración media,Gasto en publicidad por farmacia,Inventario total,Valor agregado por farmacia,Variación de existencias,Depreciación por farmacia,Eficiencia en costos,Eficiencia,Cluster
1126,2018,yucatan,yaxkukul,3,0.195,0.003,0.578,0.092,0.386,0.000,...,0.192667,0.037667,0.000000,0.0185,0.065000,0.173010,0.000000,47.179487,1300.189698,0.0
270,2018,guerrero,xalpatlahuac,3,0.479,0.011,1.378,0.037,0.931,0.001,...,0.459333,0.019333,0.000333,0.1860,0.159667,2.031930,0.000333,7.724426,869.363933,NaN
1045,2018,veracruz de ignacio de la llave,ixhuatlan del sureste,8,5.050,0.144,13.206,0.625,8.271,0.023,...,1.650750,0.292000,0.002750,0.3045,0.631250,0.810238,0.002875,12.376238,700.218293,NaN
1078,2018,veracruz de ignacio de la llave,sayula de aleman,7,7.750,0.272,16.624,0.632,8.770,0.057,...,2.374857,0.069857,0.000714,0.5505,1.107143,-0.715833,0.008143,8.154839,574.249434,19.0
936,2018,tamaulipas,abasolo,4,4.236,0.157,11.868,0.752,7.588,0.015,...,2.967000,0.000000,0.000000,0.1700,1.059000,-0.370745,0.003750,17.752597,541.882020,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,2018,puebla,honey,6,0.488,1.705,1.506,0.070,0.995,0.093,...,0.251000,0.020500,0.000000,0.1640,0.081333,-1.593625,0.015500,14.344262,4.558061,0.0
252,2018,guerrero,olinala,8,0.948,3.064,3.546,0.260,2.602,0.217,...,0.443250,0.067250,0.000125,0.5880,0.118500,0.112803,0.027125,27.426160,3.471271,NaN
703,2018,oaxaca,santa lucia ocotlan,3,0.082,0.190,0.805,0.014,0.698,0.010,...,0.268333,0.011333,0.000000,0.1535,0.027333,-3.105590,0.003333,17.073171,3.077088,0.0
676,2018,oaxaca,san dionisio ocotepec,6,0.249,0.753,1.588,0.055,1.323,0.080,...,0.264667,0.031000,0.000167,0.1000,0.041500,-1.007557,0.013333,22.088353,1.289439,0.0


In [38]:
merged_df = merged_df.dropna()

In [39]:
average_eficiencia = merged_df['Eficiencia'].mean()
print(f"The average eficiencia is: {average_eficiencia}")

The average eficiencia is: 95.61291827518625


In [40]:

cluster_stats = merged_df.groupby('Cluster')['Eficiencia'].agg(['mean', 'max', 'min', 'count'])
cluster_stats['range'] = cluster_stats['max'] - cluster_stats['min']
print(cluster_stats)


               mean          max         min  count        range
Cluster                                                         
0.0       62.200365  1300.189698    1.289439    171  1298.900259
1.0      189.918316   279.145478  144.654976      4   134.490502
2.0       59.633960   385.543280    6.938188    122   378.605092
3.0      361.704592   422.879182  300.530003      2   122.349179
4.0      108.626065   248.594076   28.691719     31   219.902357
5.0      185.566467   189.437842  181.695092      2     7.742750
6.0      141.245639   280.314248   32.216323     16   248.097925
7.0       25.638746    25.638746   25.638746      1     0.000000
8.0      257.916018   385.811421  189.698694      3   196.112727
9.0      121.746514   172.441340   23.946950      6   148.494390
10.0      98.098209   410.489687  -78.611046     74   489.100732
11.0     114.060639   267.727160   16.444803      6   251.282356
12.0     413.804805   413.804805  413.804805      1     0.000000
13.0      71.654726   319

In [44]:

result_df = pd.DataFrame()
for cluster in merged_df['Cluster'].unique():
  cluster_data = merged_df[merged_df['Cluster'] == cluster]
  cluster_mean = cluster_data['Eficiencia'].mean()
  filtered_data = cluster_data[cluster_data['Eficiencia'] >= cluster_mean]
  if len(filtered_data) >= 4:
    result_df = pd.concat([result_df, filtered_data])
  else:
    result_df = pd.concat([result_df, cluster_data])
result_df



,Año Censal,Entidad,Municipio,UE Unidades económicas,A131A Valor agregado censal bruto (millones de pesos),Q000A Acervo total de activos fijos (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A121A Consumo intermedio (millones de pesos),A700A Total de gastos (millones de pesos),Q000B Depreciación total de activos fijos (millones de pesos),...,Ingresos por farmacia,Remuneración media,Gasto en publicidad por farmacia,Inventario total,Valor agregado por farmacia,Variación de existencias,Depreciación por farmacia,Eficiencia en costos,Eficiencia,Cluster
0,2018,aguascalientes,aguascalientes,330,619.236,149.861,2298.195,153.257,1727.476,16.386,...,6.964227,0.410145,0.012982,288.5890,1.876473,1.551130,0.049655,24.749369,301.439986,40.0
548,2018,michoacan de ocampo,morelia,569,575.905,173.541,2509.272,145.998,1984.128,20.784,...,4.409968,0.259462,0.008659,296.4775,1.012135,1.279455,0.036527,25.351056,264.040019,40.0
867,2018,san luis potosi,san luis potosi,290,651.776,136.984,2571.264,123.506,1992.245,15.875,...,8.866428,0.533748,0.012800,339.5690,2.247503,2.222642,0.054741,18.949148,307.926294,40.0
1,2018,aguascalientes,asientos,9,3.464,0.858,9.816,0.583,6.554,0.170,...,1.090667,0.054778,0.000889,0.6700,0.384889,1.915240,0.018889,16.830254,80.851523,21.0
18,2018,campeche,calkini,18,7.864,1.901,45.272,2.114,37.638,0.139,...,2.515111,0.188389,0.001444,1.0710,0.436889,0.340166,0.007722,26.881994,82.378126,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,2018,nuevo leon,monterrey,383,1036.488,411.827,4732.693,429.567,3788.641,39.058,...,12.356901,0.552428,0.061366,349.3840,2.706235,1.448520,0.101979,41.444474,421.824134,23.0
775,2018,puebla,puebla,819,942.169,345.076,5218.007,490.872,4388.811,35.226,...,6.371193,0.406016,0.033737,552.2720,1.150389,1.434992,0.043011,52.100207,347.289226,47.0
838,2018,queretaro,queretaro,540,759.015,170.916,3677.187,218.061,3017.187,17.825,...,6.809606,0.328448,0.008543,410.5105,1.405583,2.185067,0.033009,28.729472,343.066942,44.0
1115,2018,yucatan,merida,441,459.500,109.663,2784.377,219.832,2360.402,12.163,...,6.313780,0.488918,0.023025,161.6130,1.041950,0.607748,0.027580,47.841567,198.692200,44.0


In [45]:
top_50_percent_by_cluster = merged_df.groupby('Cluster').apply(lambda x: x.nlargest(max(int(len(x) * 0.5), len(x) if len(x) < 4 else 0), 'Eficiencia')).reset_index(drop=True)

In [46]:
top_50_percent_by_cluster

,Año Censal,Entidad,Municipio,UE Unidades económicas,A131A Valor agregado censal bruto (millones de pesos),Q000A Acervo total de activos fijos (millones de pesos),M000A Total de ingresos por suministro de bienes y servicios (millones de pesos),A121A Consumo intermedio (millones de pesos),A700A Total de gastos (millones de pesos),Q000B Depreciación total de activos fijos (millones de pesos),...,Ingresos por farmacia,Remuneración media,Gasto en publicidad por farmacia,Inventario total,Valor agregado por farmacia,Variación de existencias,Depreciación por farmacia,Eficiencia en costos,Eficiencia,Cluster
0,2018,yucatan,yaxkukul,3,0.195,0.003,0.578,0.092,0.386,0.000,...,0.192667,0.037667,0.000000,0.0185,0.065000,0.173010,0.000000,47.179487,1300.189698,0.0
1,2018,jalisco,cuautla,3,1.132,0.043,1.940,0.057,0.968,0.010,...,0.646667,0.075667,0.000333,0.1405,0.377333,8.298969,0.003333,5.035336,528.075520,0.0
2,2018,puebla,san sebastian tlacotepec,3,3.556,0.154,10.320,0.636,6.788,0.014,...,3.440000,0.000000,0.000000,0.2180,1.185333,0.232558,0.004667,17.885264,463.508077,0.0
3,2018,puebla,chiconcuautla,4,0.131,0.007,0.577,0.014,0.446,0.001,...,0.144250,0.000000,0.000000,0.0000,0.032750,0.000000,0.000250,10.687023,371.080831,0.0
4,2018,puebla,zacapala,3,6.324,0.386,16.362,0.794,10.027,0.037,...,5.454000,0.000000,0.000667,0.2660,2.108000,-0.061117,0.012333,12.555345,330.943514,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,2018,jalisco,zapotlan el grande,62,56.943,11.317,254.635,15.279,204.640,1.131,...,4.107016,0.293758,0.012000,32.1105,0.918435,2.439963,0.018242,26.832095,117.296403,48.0
561,2018,sonora,caborca,37,37.820,8.395,114.829,10.428,77.688,1.091,...,3.103486,0.172378,0.014486,5.0390,1.022162,0.278675,0.029486,27.572713,110.777917,48.0
562,2018,oaxaca,heroica ciudad de huajuapan de leon,49,40.397,7.869,211.617,19.090,175.409,0.925,...,4.318714,0.441714,0.019551,12.9160,0.824429,1.239031,0.018878,47.255984,110.282820,48.0
563,2018,baja california,mexicali,452,805.336,308.769,2684.417,311.258,1891.090,28.055,...,5.938976,0.382949,0.023569,116.5675,1.781717,0.218185,0.062069,38.649458,374.245686,49.0


### Exportar

In [48]:
entidad_municipio_table = top_50_percent_by_cluster[['Entidad', 'Municipio']]

entidad_municipio_table.to_csv('../Datos/Resultados/TopFarmacias2.csv', index=False)

In [50]:
entidad_municipio_table = result_df[['Entidad', 'Municipio']]

entidad_municipio_table.to_csv('../Datos/Resultados/TopFarmacias.csv', index=False)